In [1]:
## This project started on 6/21/2025. This project is meant to find the relationship of restrictive gun laws with the number of mass 
# shooting victims. Limitations that I am aware of are the supplementary effects of: population density, urban vs. rural areas, 
# population age, access to mental health care, level of poverty/cost of living that influences increase in stress, and quality of education.
# Resources: Gun Violence Archive dataset for mass shooting stats, JoshData Census dataset and Census.gov for population by state by year

!pip install --upgrade plotly
!pip install pandas openpyxl

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

# Change current working directory to desired location.
# os.chdir(r"C:\Users\toddz\OneDrive\Documents\Coding Things\Personal Projects\Mass Shootings Data Project")
# new_directory = os.getcwd()
# new_directory
os.getcwd()

'C:\\Users\\toddz\\OneDrive\\Documents\\Coding Things\\Personal Projects\\Mass Shootings Data Project'

In [3]:
# Creates a data frame using Gun Violence Archive .csv file
df_gva = pd.read_csv('Gun Violence Archive Dataset.csv')

# Creates dictionary of full name states to abbreviations ('Alaska' to 'AK')
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Replaces full name states with abbrevations
df_gva['State'] = df_gva['State'].map(state_abbrev)

# Replaces 'dd-mon-yy' format of GVA dataset into three separate columns for day month and year
df_gva['Incident Date'] = pd.to_datetime(df_gva['Incident Date'], format='%d-%b-%y')
df_gva['Day'] = df_gva['Incident Date'].dt.day
df_gva['Month'] = df_gva['Incident Date'].dt.month
df_gva['Year'] = df_gva['Incident Date'].dt.year

df_gva

,Incident ID,Incident Date,State,City Or County,Address,Victims Killed,Victims Injured,Suspects Killed,Suspects Injured,Suspects Arrested,Operations,Day,Month,Year
0,3242052,2025-06-20,WI,Milwaukee,8300 block of W Brown Deer Rd,1,4,0,0,0,NaN,20,6,2025
1,3240121,2025-06-19,OK,Tulsa,S Denver Ave and W 15th St,0,5,0,0,0,NaN,19,6,2025
2,3239589,2025-06-17,MS,Louisville,200 block of W College St,1,3,0,0,1,NaN,17,6,2025
3,3238923,2025-06-17,WI,Milwaukee,1000 block of W Mineral St,0,4,0,0,0,NaN,17,6,2025
4,3238015,2025-06-16,OH,Cleveland,5900 block of Luther Ave,1,6,0,0,0,NaN,16,6,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5343,95500,2014-01-12,LA,Delhi,3600 block of US-80 W,0,6,0,0,8,NaN,12,1,2014
5344,95146,2014-01-11,MS,Jackson,3430 W. Capitol Street,0,4,0,0,0,NaN,11,1,2014
5345,94514,2014-01-05,PA,Erie,829 Parade St,1,3,0,0,0,NaN,5,1,2014
5346,92704,2014-01-03,NY,Queens,Farmers Boulevard and 133rd Avenue,1,3,0,0,0,NaN,3,1,2014


In [4]:
# Creates a column, 'Total Victims' to use in overall shootings per million calculation
df_gva['Total Victims'] = df_gva['Victims Injured'] + df_gva['Victims Killed']

# Sets dataframe as indices for reorganization
df_gva.columns = range(df_gva.shape[1])

# Sets up dataframe in logical sequence
df_gva = df_gva[[11,12,13,2,14]]
df_gva.head()

,11,12,13,2,14
0,20,6,2025,WI,5
1,19,6,2025,OK,5
2,17,6,2025,MS,4
3,17,6,2025,WI,4
4,16,6,2025,OH,7


In [5]:
# Renames headers to appropriate columns
df_gva.columns = ['Day', 'Month', 'Year', 'State', 'Total Victims']
df_gva.head()

,Day,Month,Year,State,Total Victims
0,20,6,2025,WI,5
1,19,6,2025,OK,5
2,17,6,2025,MS,4
3,17,6,2025,WI,4
4,16,6,2025,OH,7


In [6]:
# loads JoshData census data as separate dataframe
df_jd = pd.read_csv('JoshData Census 1950-2019.csv')

# Makes all headers index '0' through '2'
df_jd.columns = range(df_jd.shape[1])

# Makes column headers 'State', 'Year', and 'Population'
df_jd.columns = ['State', 'Year', 'Population']
df_jd.head()

,State,Year,Population
0,AK,1951,158000
1,AK,1952,189000
2,AK,1953,205000
3,AK,1954,215000
4,AK,1955,222000


In [7]:
# Cleans JoshData dataset to remove all years earlier than 2014 (dataset is now 2014-2024)
df_jd_filt = df_jd[df_jd['Year'] >= 2014]
df_jd_filt.head()

,State,Year,Population
63,AK,2014,737075
64,AK,2015,738430
65,AK,2016,742575
66,AK,2017,740983
67,AK,2018,736624


In [8]:
# Merges both JoshData dataset and GVA dataset to add population column
merged_df_gva = pd.merge(df_gva, df_jd_filt, on=['State', 'Year'], how='left')

# Removes all null or NaN cells
merged_df_gva.dropna(inplace=True)
merged_df_gva = merged_df_gva[merged_df_gva["Population"] != ""]

merged_df_gva

,Day,Month,Year,State,Total Victims,Population
165,31,12,2024,CA,4,39431263.0
166,31,12,2024,MS,6,2943045.0
167,30,12,2024,NY,6,19867248.0
168,30,12,2024,NY,4,19867248.0
169,28,12,2024,WV,4,1769979.0
...,...,...,...,...,...,...
5343,12,1,2014,LA,6,4645938.0
5344,11,1,2014,MS,4,2991892.0
5345,5,1,2014,PA,4,12792392.0
5346,3,1,2014,NY,4,19653431.0


In [9]:
# Sums the daily values by year for each state
grouped_df_gva = merged_df_gva.groupby(['Year', 'State', 'Population'])[['Total Victims']].sum().reset_index()

grouped_df_gva

,Year,State,Population,Total Victims
0,2014,AK,737075.0,6
1,2014,AL,4843737.0,10
2,2014,AR,2968759.0,11
3,2014,AZ,6732873.0,4
4,2014,CA,38586706.0,191
...,...,...,...,...
431,2024,UT,3503613.0,10
432,2024,VA,8811195.0,58
433,2024,WA,7958180.0,28
434,2024,WI,5960975.0,51


In [10]:
# Adds a column of Victims per Million
grouped_df_gva['Victims per Million'] = grouped_df_gva['Total Victims']*1000000 / grouped_df_gva['Population']
grouped_df_gva

,Year,State,Population,Total Victims,Victims per Million
0,2014,AK,737075.0,6,8.140284
1,2014,AL,4843737.0,10,2.064522
2,2014,AR,2968759.0,11,3.705252
3,2014,AZ,6732873.0,4,0.594100
4,2014,CA,38586706.0,191,4.949891
...,...,...,...,...,...
431,2024,UT,3503613.0,10,2.854197
432,2024,VA,8811195.0,58,6.582535
433,2024,WA,7958180.0,28,3.518392
434,2024,WI,5960975.0,51,8.555647


In [11]:
# Adds the RAND Corporation's Firearm Law database for determining the relationship of number of Restrictive laws on Victims per Million
df_rand = pd.read_excel('RAND Firearm Law Database.xlsx', sheet_name='Database')
df_rand

,Law ID,State,State Postal Abbreviation,FIPS Code,Law Class (num),Law Class,Law Class Subtype,Handguns or Long Guns,Effect,Type of Change,...,Content,Supersession Date Year,Supersession Date Month,Supersession Date Day,Age for Minimum Age Laws,"Length of Waiting Period (days, handguns)",Surrender Authorized,Additional Context and Notes,Discrepancies,Exception Code
0,AK1002,Alaska,AK,2,2,carrying a concealed weapon (ccw),prohibited,handgun,Restrictive,Implement,...,It is unlawful for a person to carry concealed...,1994.0,10.0,1.0,NaN,NaN,NaN,Prior law prohibiting concealed carry enacted ...,NaN,NaN
1,AK1003,Alaska,AK,2,2,carrying a concealed weapon (ccw),shall issue,handgun,Permissive,Modify,...,Sec. 18.65.700. Permit to carry a concealed ha...,2003.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AK1004,Alaska,AK,2,2,carrying a concealed weapon (ccw),shall issue (permit not required),handgun,Permissive,Modify,...,§ 11.61.220(a) A person commits the crime of m...,NaN,NaN,NaN,NaN,NaN,NaN,Permitting system maintained for residents see...,NaN,NaN
3,AK1005,Alaska,AK,2,3,castle doctrine,NaN,handgun and long gun,Permissive,Modify,...,b) A person may not use deadly force under thi...,2006.0,9.0,13.0,NaN,NaN,NaN,See 2006 S.B. No. 200 Ch. 68.,NaN,NaN
4,AK1006,Alaska,AK,2,3,castle doctrine,expanded 2,handgun and long gun,Permissive,Modify,...,A person may not use deadly force under this s...,2013.0,9.0,18.0,NaN,NaN,NaN,Catagorized as expanded 2 because removes duty...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,WY1053,Wyoming,WY,56,6,firearm sales restrictions,assault weapons ban — Federal,long gun,Restrictive,Implement,...,This subtitle may be cited as the “Public Safe...,2004.0,9.0,13.0,NaN,NaN,NaN,This subtitle and the amendments made by this ...,NaN,NaN
1960,WY1054,Wyoming,WY,56,6,firearm sales restrictions,ban on high capacity magazines – federal,handgun and long gun,Restrictive,Implement,...,SEC. 110103. BAN OF LARGE CAPACITY AMMUNITION ...,2004.0,9.0,13.0,NaN,NaN,NaN,This subtitle and the amendments made by this ...,NaN,NaN
1961,WY1055,Wyoming,WY,56,7,minimum age,minimum age purchase - federal,long gun,Restrictive,Implement,...,(b) It shall be unlawful for any licensed impo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962,WY1056,Wyoming,WY,56,7,minimum age,minimum age purchase - federal,handgun,Restrictive,Implement,...,(x)(1) It shall be unlawful for a person to se...,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN


In [12]:
# Filters out database to show only the 'Type of Change' column = 'Implement' & 'Modify';  'Effect' column = 'Restrictive' for 2014-2024
df_rand_filt = df_rand[df_rand['Effect'] == 'Restrictive']
df_rand_filt = df_rand_filt[df_rand_filt['Type of Change'] != 'Repeal']
df_rand_filt = df_rand_filt[df_rand_filt['Effective Date Year'] > 1950]

# Numbers the index of all columns for trimming only to State, Effect, Type of Change, Year, and Restrictive laws Added/Modified columns
df_rand_filt.columns = range(df_rand_filt.shape[1])
df_rand_filt = df_rand_filt[[2,11,8]]
df_rand_filt.columns = ['State', 'Year', 'Effect']

# Denotes each cell with restrictive law as '1' for summing
df_rand_filt['Effect'] = 1
df_rand_filt

,State,Year,Effect
0,AK,1978,1
5,AK,1980,1
6,AK,1992,1
7,AK,1980,1
8,AK,1992,1
...,...,...,...
1959,WY,1994,1
1960,WY,1994,1
1961,WY,1994,1
1962,WY,1994,1


In [13]:
#Creates a base dataframe of all 50 states (and DC) each year from 2014-2024 for merging with df_rand_filt
all_states = df_rand['State Postal Abbreviation'].dropna().unique()
all_years = list(range(1950,2025))
df_none = pd.MultiIndex.from_product([all_states, all_years], names=['State', 'Year']).to_frame(index=False)

df_rand_full = pd.merge(df_none, df_rand_filt, on=['State', 'Year'], how='left')
df_rand_full

,State,Year,Effect
0,AK,1950,NaN
1,AK,1951,NaN
2,AK,1952,NaN
3,AK,1953,NaN
4,AK,1954,NaN
...,...,...,...
4763,PA,2020,NaN
4764,PA,2021,NaN
4765,PA,2022,NaN
4766,PA,2023,NaN


In [14]:
# Properly formats Number of Restrictive Laws Added/Modified column to show sum of restrictive laws for each state for each year
grouped_df_rand = df_rand_full.copy() # Creates a copy dataframe to avoid modifying original
grouped_df_rand = grouped_df_rand.groupby(['State', 'Year'])['Effect'].sum().reset_index()
grouped_df_rand['Effect'] = grouped_df_rand.groupby('State')['Effect'].cumsum()

# Renames 'Effect' Column to 'Restrictive Laws Added/Modified'
grouped_df_rand.rename(columns={'Effect':'Restrictive Laws Added/Modified Since 1950'}, inplace=True)

# Sorts dataframe by year ascending
sorted_df_rand = grouped_df_rand.sort_values(by=['Year'], ascending=True)
sorted_df_rand

,State,Year,Restrictive Laws Added/Modified Since 1950
0,AK,1950,0.0
1500,MD,1950,0.0
2325,NJ,1950,0.0
1875,MS,1950,0.0
1950,MT,1950,0.0
...,...,...,...
749,FL,2024,31.0
2024,MT,2024,11.0
1799,MN,2024,26.0
1499,MA,2024,50.0


In [15]:
# Merges cleaned Rand dataframe and cleaned GVA dataframe
merged_df_rand = pd.merge(sorted_df_rand, grouped_df_gva, on=['State', 'Year'], how='left')
merged_df_rand

,State,Year,Restrictive Laws Added/Modified Since 1950,Population,Total Victims,Victims per Million
0,AK,1950,0.0,NaN,NaN,NaN
1,MD,1950,0.0,NaN,NaN,NaN
2,NJ,1950,0.0,NaN,NaN,NaN
3,MS,1950,0.0,NaN,NaN,NaN
4,MT,1950,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
3820,FL,2024,31.0,23372215.0,160.0,6.845735
3821,MT,2024,11.0,NaN,NaN,NaN
3822,MN,2024,26.0,5793151.0,42.0,7.249940
3823,MA,2024,50.0,7136171.0,32.0,4.484197


In [18]:
## Creates chloropleth map of states with higher victims per million as well as a bar graph below the map showing the trend from 2014-2024.
# The graph includes a single green line showing the cumulative number of gun laws implemented in that state since 1950 for comparison.

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot
from ipywidgets import Output
import plotly.io as pio

# Creates the initial figure
fig = make_subplots(
    rows=2, cols=1,
    specs=[[{"type": "choropleth"}], [{"type": "xy"}]],
    vertical_spacing=0.1,
    subplot_titles=("US Gun Violence by State", "State-Specific Yearly Trend (2014-2024)")
)

# Makes the chloropleth for Victims per Million
fig.add_trace(
    go.Choropleth(
        locations=merged_df_rand['State'],
        z=merged_df_rand['Victims per Million'],
        zmin=0,
        zmax=30,
        locationmode='USA-states',
        colorscale='Blues',
        colorbar_title="Victims per Million",
        hovertemplate='%{location}: %{z}<extra></extra>'
    ),
    row=1, col=1
)

# Makes the bar graph
fig.add_trace(
    go.Bar(x=[], y=[], name="Yearly Data"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=[], y=[], mode="lines", name="Restrictive Gun Laws Since 1950"),
    row=2, col=1
)

fig.update_layout(
    height=800,
    geo=dict(
        scope='usa',
        projection=go.layout.geo.Projection(type='albers usa'),
    )
)

# Add click event callback
fig.update_layout(
    clickmode='event+select'
)

# Create an empty FigureWidget from the figure
fig_widget = go.FigureWidget(fig)
output = Output()
fig_widget.update_yaxes(range=[merged_df_rand['Victims per Million'], merged_df_rand['Restrictive Laws Added/Modified Since 1950']], row=2, col=1)
fig_widget.update_xaxes(range=[2014,2024], row=2, col=1)
fig_widget.update_yaxes(title_text="Victims per Million", row=2, col=1)

# Define the callback function for click events
@output.capture()
def update_bar(trace, points, selector):
    if points.point_inds:
        # Get the clicked state
        clicked_state = fig_widget.data[0].locations[points.point_inds[0]]
        
        # Filter data for the selected state
        state_data = merged_df_rand[merged_df_rand['State'] == clicked_state]
        
        # Update the bar chart for victims per million 
        fig_widget.data[1].x = state_data['Year']
        fig_widget.data[1].y = state_data['Victims per Million']
        fig_widget.data[1].name = f"{clicked_state} Yearly Trend"

        # Update the line plot on the bar chart for number of restrictive gun laws since 1950
        fig_widget.data[2].x = state_data['Year']
        fig_widget.data[2].y = state_data['Restrictive Laws Added/Modified Since 1950']
        fig_widget.data[2].name = f"{clicked_state} Restrictive Gun Laws Since 1950"
        
        # Update subtitle
        fig_widget.layout.annotations[1].text = f"Yearly Trend for {clicked_state}"

# Connect the callback to the choropleth trace
fig_widget.data[0].on_click(update_bar)

# Display the figure widget
display(fig_widget)
display(output)

C:\Users\toddz\anaconda3\Lib\site-packages\jupyter_client\session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Victims per Million'}},
              'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                             'rgb(222,235,247)'], [0.25, 'rgb(198,219,239)'],
                             [0.375, 'rgb(158,202,225)'], [0.5,
                             'rgb(107,174,214)'], [0.625, 'rgb(66,146,198)'],
                             [0.75, 'rgb(33,113,181)'], [0.875, 'rgb(8,81,156)'],
                             [1.0, 'rgb(8,48,107)']],
              'geo': 'geo',
              'hovertemplate': '%{location}: %{z}<extra></extra>',
              'locationmode': 'USA-states',
              'locations': array(['AK', 'MD', 'NJ', ..., 'MN', 'MA', 'WY'], dtype=object),
              'type': 'choropleth',
              'uid': '1076997e-f1a2-4171-b099-03a513d9a57e',
              'z': {'bdata': ('AAAAAAAA+H8AAAAAAAD4fwAAAAAAAP' ... 'dbZvD/HECNZLF20e8RQAAAAAAAAPh/'),
                    'dtype': 'f8'},
              'zmax': 30,
    

Output()